## Data-preparatie

Dit notebook behandelt de data-preparatie en geeft antwoord op deelvraag 1:

*"Hoe kan cellijn-data (mutaties, methylatie, drug-respons) worden verzameld, opgeschoond en geschikt gemaakt voor machine learning?"*

Voor dit onderzoek worden drie afzonderlijke datasets gebruikt die afkomstig zijn van het CCLE-project (Cancer Cell Line Encyclopedia). De data beschrijven verschillende moleculaire kenmerken van kankercellijnen:

- OmicsSomaticMutations.csv - bevat de somatische mutatieprofielen van de cellijnen.
- CCLE_RRBS_TSS1kb_20181022.txt - bevat DNA-methylatiegegevens gemeten rond de Transcription Start Site (TSS, ±1 kb). Dit gebied omvat zowel promoterregio's als het eerste exon, die belangrijk zijn voor genregulatie.
- Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv - bevat therapieresponsgegevens (AUC-waarden) die aangeven hoe gevoelig een cellijn is voor een bepaald medicijn.

Het uiteindelijke doel is om een dataset te creëren waarin elke longkankercellijn zowel moleculaire kenmerken (mutaties en methylatie) als een responswaarde (AUC) bevat, zodat deze kan worden gebruikt als input voor supervised machine learning.

**xxx** toelichting CpG sites en avg coverage (kwaliteitsmaat) xxx:
CpG_sites_hg19 = genomische voordinaten van CpG-sites in het Hg19-genoom
avg_coverage = maat voor de betrouwbaarheid van de methylatie-metingen op dat locus

TSS=Transcription Start Site, het beginpunt van een gen waar transcriptie start.
±1kb betekent dat de gemeten regio 1 kilobase (1000bp) upstream en downstream van de TSS ligt
dus direct rond het begin van het gen, inclusief een stukje voor (promotorgebied) en erna (eerste exon/regio vlakbij TSS)

- Upstream van TSS (~1000 bp): dit is meestal de promoter. Methylatie hier kan de transcriptie van het gen onderdrukken.
- Downstream van TSS (~1000 bp): dit is vaak het 5’ einde van het gen, inclusief exon 1 of de 5’ UTR. Methylatie hier kan ook effect hebben op genexpressie. methylatie in deze regio kan het initiatiefase van transcriptie veranderen, waardoor het gen minder of anders wordt afgelezen

Nadenken over avg_coverage. minimum instellen. te lagen waarden weglaten omdat de methylatiepercentages dan ruis bevatten (in de literatuur 10>)
laat ook zien want de impact is van die drempel.


#### Stap 1. Data verkennen

Doel: inzicht krijgen in de structuur van elk bestand voor het filteren en samenvoegen.

Het is belangrijk om de kolomnamen, rijenstructuur en datatypes te begrijpen. Zo krijg je inzicht in welke kolommen relevant zijn (bijv. cell_line_id, gene, AUC, avg_coverage) en kun je bepalen hoe de datasets aan elkaar gekoppeld kunnen worden.

stappen:
Controleer het aantal rijen/kolommen.
Bekijk een paar voorbeeldrijen om de structuur te snappen.
Noteer de naam van de kolom die de cellijn identificeert (ID).
Kijk of de kolomnamen consistent zijn tussen bestanden.
Kijk hoeveel unieke cellijnen er per bestand zijn.))


In [ ]:
import pandas as pd

# aanmaken variabele met de bestandpad
mutatie_file = "../data/raw/OmicsSomaticMutations.csv"

# inlezen 
mutatie_df = pd.read_csv(mutatie_file) # csv, geen tabs

# controleer eerste paar rijen
print(mutatie_df.head())


In [ ]:
# aanmaken variabele met de bestandpad
methylatie_file = "../data/raw/CCLE_RRBS_TSS1kb_20181022.txt"

# inlezen
methylatie_df = pd.read_csv(methylatie_file, sep='\t') # txt, tab seperated

# controleer eerste paar rijen
print(methylatie_df.head())

In [ ]:
# aanmaken variabele met de bestandpad
response_file = "../data/raw/Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv"

# inlezen
response_df = pd.read_csv(response_file) # csv, geen tabs

# controleer eerste paar rijen
print(response_df.head())

#### Stap 2. Filteren op relevante data

Doel: alleen de gegevens behouden die biologisch en analytisch relevant zijn.

De datasets worden beperkt tot de relevante subset (longkankercellijnen). Zo wordt voorkomen dat je een hele grote, onoverzichtelijke tabel krijgt met onnodige kolommen. Daarnaast bespaart het rekentijd.

stappen:
Filter cellijnen op weefseltype “lung”.
Selecteer alleen de features/kolommen die je nodig hebt (bijv. genmutaties, methylatiepercentage, AUC).
Voor methylatie: filter CpG-sites met avg_coverage < 10 om meetruis te verminderen — leg uit dat deze drempel in de literatuur wordt gebruikt om betrouwbaarheid te waarborgen (voeg bron toe)

In [2]:
#

#### Stap 3. Opschonen

Doel: ervoor zorgen dat alle bestanden dezelfde structuur en sleutel-IDs gebruiken.

De kolomnamen moeten worden gestandaardiseerd. Dit houd in dat alledrie de bestanden dezelfde sleutel-IDs moeten krijgen: cell_line_id. Ook moeten dubbele rijen of niet-unieke cellijnen moeten worden verwijderd. Er moet gecontroleerd worden op ontbrekende waarden (NaN) in de belangrijke kolommen. Dit is belangrijk om fouten te voorkomen bij het samenvoegen.

**xxx** avc_covverage in deze stap? > Waarden lager dan 10 worden verwijderd, omdat deze metingen onvoldoende dekking hebben om betrouwbare methylatiegegevens te geven. Dit voorkomt dat technische ruis invloed heeft op het model.

#### Stap 4. Transformatie van features

Doel: 
- mutatiegegevens omzetten naar binaire variabelen (0=mutatie, 1=mutatie).
- methylatiegegevens reduceren/aggregeren (gemiddelde pakken?)


Machine-learningmodellen werken met numerieke inputs; binaire encoding wordt veelgebruikt.

Methylatie heeft vaak duizenden features wat kan leiden tot overfitting? **meer uitleg nodig, bron**

Wanneer er meerdere AUC-metingen zijn per cellijn moet het gemiddelde worden berekend.

#### Stap 5. Samenvoegen 

Doel: het creëren van één tabel waarin elke rij één cellijn is, met alle relevante features en bijbehorende responswaarde.

De drie datasets worden samengevoegd op de gemeenschappelijke sleutel cell_line_id. 

Er wordt gebruik gemaakt van een **inner join** om ontbrekende waarden in de uiteindelijke tabel te voorkomen. Alleen cellijnen waarvan zowel moleculaire data (mutaties, methylatie) als responsdata beschikbaar zijn, worden behouden. Dit is een belangrijke eis voor supervised learning.

check ook hoeveel cellijnen verloren gaan door die joing.


#### Stap 6. Controle op ontbrekende waarden

Doel: controleren of de uiteindelijke dataset compleet genoeg is.

Cellijnen met onbrekende AUC moeten worden verwijderd omdat machine-learningalgoritmen niet kunnen omgaan met ontbrekende waarden. 



#### Stap 7. Preprocessing voor machine learning

Doel: de data omzetten naar numerieke vorm, schalen en voorbereiden voor modellering.

**hier start scikit-learn** Pandas is vooral geschikt voor het inlezen, bewerken en samenvoegen van dataframes. Scikit-learn is gespecialiseerd in preprocessing. 

**xxx** verwijs naar scikit-learn tutorial?




